<a href="https://colab.research.google.com/github/lucas-fpaiva/estatistica_pes/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%reset -f

#Bibliotecas

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import random as python_random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
#IA
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import GRU

import warnings
warnings.filterwarnings('ignore')
% matplotlib inline

#Funções

In [4]:
def reset_seeds():
   np.random.seed(4) 
   python_random.seed(9498)
   tf.random.set_seed(9)

reset_seeds() 

def data_import(txt):
  global columns
  columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21"]
  df = pd.read_csv("/content/drive/MyDrive/cmap/"+txt, sep= "\s+", header = None,names=columns )
  return df

def add_rul(g):
    g['RUL'] = max(g['cycle']) - g['cycle']
    return g

def add_cycle2(g):
    g['cycle2'] = 362 - g['RUL']
    return g

def new_data(train_data):
        new_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
        new_data['sensor']=[columns[5]]*train_data.shape[0]
        new_data['sensor_value']= train_data[columns[5]]
        for column in columns[5:]:
            temp_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
            temp_data['sensor']=[column]*train_data.shape[0];
            temp_data['sensor_value']= train_data[column];
            new_data = pd.concat([new_data,temp_data],ignore_index=True)
        return new_data 

def Suavisar(train_data, mm):
        new_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
        new_data['sensor_value']= train_data[columns[5]]
        for column in columns[5:]:
            temp_data = train_data[["id","cycle","op1","op2","op3","RUL"]]
            temp_data['sensor']=[column]*train_data.shape[0];
            temp_data['sensor_value']= train_data[column];
            new_data = pd.concat([new_data,temp_data],ignore_index=True)
        return new_data 


#Dados

In [5]:
train_FD001 = data_import("train_FD001.txt") 
train_FD002 = data_import("train_FD002.txt") 
train_FD003 = data_import("train_FD003.txt") 
train_FD004 = data_import("train_FD004.txt") 

train_FD001 = train_FD001.groupby('id').apply(add_rul)
train_FD002 = train_FD002.groupby('id').apply(add_rul)
train_FD003 = train_FD003.groupby('id').apply(add_rul)
train_FD004 = train_FD004.groupby('id').apply(add_rul)

train_FD001_2 = train_FD001.groupby('id').apply(add_cycle2)
train_FD002_2 = train_FD002.groupby('id').apply(add_cycle2)
train_FD003_2 = train_FD003.groupby('id').apply(add_cycle2)
train_FD004_2 = train_FD004.groupby('id').apply(add_cycle2)

new_train_FD001 = new_data(train_FD001) 
new_train_FD002 = new_data(train_FD002) 
new_train_FD003 = new_data(train_FD003) 
new_train_FD004 = new_data(train_FD004) 

In [6]:
max_list = []
def add_rul(g):
    global max_list
    max_list.append(max(g['cycle']))
    return g

r = train_FD001.groupby('id').apply(add_rul)

In [7]:
dataset = train_FD001[["RUL","sensor2","sensor3","sensor4","sensor7","sensor12","sensor15","sensor17","sensor20","sensor21"]]

In [8]:
scaler = MinMaxScaler(feature_range=(-1, 1))
x = scaler.fit_transform(dataset[["sensor2","sensor3","sensor4","sensor7","sensor12","sensor15","sensor17","sensor20","sensor21"]])
y = scaler.fit_transform(dataset[["RUL"]])

In [9]:
x = scaler.fit_transform(dataset[["sensor2","sensor3","sensor4","sensor7","sensor12","sensor15","sensor17","sensor20","sensor21"]])

In [10]:
look_back = 30
future = 1
k=0
X_train = []
y_train = []
for j in max_list:
  print()
  for i in range(look_back+k, k+j-future):
    X_train.append(x[i-look_back:i])
    y_train.append(y[i])
  k=k+j
X_train, y_train = np.array(X_train), np.array(y_train)

#Modelo

In [11]:
regressor = Sequential()
regressor.add(LSTM(units =128, return_sequences = True, input_shape = (X_train.shape[-2:])))
regressor.add(LSTM(units = 32))
regressor.add(Dense(units = 1, activation='LeakyReLU'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping#4
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history = regressor.fit(X_train, y_train, epochs = 100, batch_size = 1024, validation_split=0.2,callbacks=[early_stop])

Epoch 1/100
14/14 [==============================] - 14s 687ms/step - loss: 0.2627 - val_loss: 0.1606
Epoch 2/100
14/14 [==============================] - 8s 602ms/step - loss: 0.0588 - val_loss: 0.0965
Epoch 3/100
14/14 [==============================] - 8s 605ms/step - loss: 0.0486 - val_loss: 0.0873
Epoch 4/100
14/14 [==============================] - 8s 606ms/step - loss: 0.0461 - val_loss: 0.0832
Epoch 5/100
14/14 [==============================] - 9s 657ms/step - loss: 0.0442 - val_loss: 0.0823
Epoch 6/100
14/14 [==============================] - 8s 604ms/step - loss: 0.0417 - val_loss: 0.0775
Epoch 7/100
14/14 [==============================] - 10s 663ms/step - loss: 0.0418 - val_loss: 0.0673
Epoch 8/100
14/14 [==============================] - 8s 594ms/step - loss: 0.0375 - val_loss: 0.0669
Epoch 9/100
14/14 [==============================] - 8s 596ms/step - loss: 0.0331 - val_loss: 0.0575
Epoch 10/100
14/14 [==============================] - 8s 601ms/step - loss: 0.0329 - val_

#Testes

In [ ]:
columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21"]


test_data = pd.read_csv("/content/drive/MyDrive/cmap/test_FD001.txt", sep= "\s+", header = None,names=columns)

In [ ]:
max_list = []
def add_rul(g):
    global max_list
    max_list.append(max(g['cycle']))
    return g

r = test_data.groupby('id').apply(add_rul)

In [ ]:
x2 = scaler.transform(test_data[["sensor2","sensor3","sensor4","sensor7","sensor12","sensor15","sensor17","sensor20","sensor21"]])

In [ ]:
look_back = 30
future = 1
k=0
X_test = []
for j in max_list:
  X_test.append(x2[j+k-look_back:j+k])
  k=k+j
X_test = np.array(X_test)

In [ ]:
resultado = regressor.predict(X_test)
y = scaler.fit_transform(dataset[["RUL"]])
resultado  = scaler.inverse_transform(resultado)


In [ ]:
y_test = pd.read_csv("/content/drive/MyDrive/cmap/RUL_FD001.txt", sep= "\s+", header = None)
p=resultado
y_test = y_test.values
p=np.reshape(p, len(p))
y=np.array(y_test)
y=np.reshape(y_test, len(y_test))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error,mean_absolute_error

In [ ]:
mean_absolute_error(y,p)

In [ ]:
MAE = []
RMSE = []
for i in range(100):
  MAE.append(mean_absolute_error(y[0+i:i+1],p[0+i:i+1]))
  RMSE.append(np.sqrt(mean_squared_error(y[0+i:i+1],p[0+i:i+1])))

In [ ]:
RMSE_new = np.sqrt(mean_squared_error(y,p))
print("RMSE (Taking only last examples): ", RMSE_new)

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
#p,y_test = zip(*sorted(zip(p,y_test)))
fig = go.Figure()
fig.add_trace(go.Scatter(y=p,name="previsão"))#,mode='markers',marker_symbol="x"))
fig.add_trace(go.Scatter(y=y,name="Dados"))#,mode='markers'))
fig.update_layout(title='Dados vs Previsão',
                   xaxis_title='Unidade',
                   yaxis_title='RUL')
fig.show()

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
#p,y_test = zip(*sorted(zip(p,y_test)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=y,y=p,mode='markers'))
fig.update_layout(title='Dados vs Previsão',
                   xaxis_title='Dados',
                   yaxis_title='Previsão',
                  width=700,height=700)
fig.show()

In [ ]:
regressor.save("LSTM_Mult.h5")